In [ ]:
!nvidia-smi

Fri Jun  6 10:19:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!python --version

Python 3.11.13


In [ ]:
import cupy

print(cupy.__version__)

cuda_version = cupy.cuda.runtime.runtimeGetVersion()
print("CUDA Runtime Version:", cuda_version)

13.3.0
CUDA Runtime Version: 12060


In [ ]:
!pip install spacy[lookups]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.7 MB/s eta 0:00:00


In [ ]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Check if spaCy can run on GPU
spacy.require_gpu()

True

In [ ]:
nlp = spacy.load("pt_core_news_lg")

In [ ]:
text = "Steve Wozniak, CEO da Apple, lançou no novo iPhone em Cupertino, Califórnia."

In [ ]:
doc = nlp(text)

In [ ]:
print("POS Tagging:")
for token in doc:
    print(f"{token.text} - {token.pos_} ({token.tag_})")

POS Tagging:
Steve - PROPN (PROPN)
Wozniak - PROPN (PROPN)
, - PUNCT (PUNCT)
CEO - PROPN (PROPN)
da - ADP (ADP)
Apple - PROPN (PROPN)
, - PUNCT (PUNCT)
lançou - VERB (VERB)
no - ADP (ADP)
novo - ADJ (ADJ)
iPhone - NOUN (NOUN)
em - ADP (ADP)
Cupertino - PROPN (PROPN)
, - PUNCT (PUNCT)
Califórnia - PROPN (PROPN)
. - PUNCT (PUNCT)


In [ ]:
# NER
print("\nNamed Entities (NER):")
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")


Named Entities (NER):
Steve Wozniak - PER
CEO - ORG
Apple - ORG
iPhone - MISC
Cupertino - LOC
Califórnia - LOC


In [ ]:
# Visualizar a árvore de dependência
displacy.render(doc, style="dep", jupyter=True)

# Combinando Named Entity Recognition com Dependências sintáticas do texto

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter
import numpy as np

In [ ]:
texts = [
    "O presidente discursou na ONU.",
    "A Apple lançou o novo iPhone no Brasil.",
    "O Brasil venceu a Copa América em 2021.",
    "A Google está investindo em inteligência artificial.",
    "O governo do Brasil anunciou novas medidas econômicas."
]

In [ ]:
# Extração e contagem de dependências em uma frase
def extract_dependency_counts(doc):
    dep_counts = Counter([token.dep_ for token in doc])
    return dep_counts

In [ ]:
# Extração e contagem de NERs em uma frase
def extract_ner_counts(doc):
    ner_counts = Counter([ent.label_ for ent in doc.ents])
    return ner_counts

In [ ]:
# Analise dos textos, extração e contagem de NERs e dependências
dep_features = []
ner_features = []
for text in texts:
    doc = nlp(text)
    dep_counts = extract_dependency_counts(doc)
    ner_counts = extract_ner_counts(doc)
    dep_features.append(dep_counts)
    ner_features.append(ner_counts)

In [ ]:
# Criação de um conjunto único de relações de dependência e labels NER para vetorização
all_dependencies = set(dep for dep_count in dep_features for dep in dep_count.keys())
all_ner_labels = set(ner for ner_count in ner_features for ner in ner_count.keys())

In [ ]:
# converte a contagem de dependências em uma matriz
dep_features_matrix = []
for dep_count in dep_features:
    # Create a vector based on the presence/frequency of each dependency relation
    vector = [dep_count.get(dep, 0) for dep in all_dependencies]
    dep_features_matrix.append(vector)

In [ ]:
# Converte a contagem de NERs em uma matriz
ner_features_matrix = []
for ner_count in ner_features:
    # Create a vector based on the presence/frequency of each NER type
    vector = [ner_count.get(ner, 0) for ner in all_ner_labels]
    ner_features_matrix.append(vector)

In [ ]:
# Converte ambas as martizes em array do numpy
dep_features_matrix = np.array(dep_features_matrix)
ner_features_matrix = np.array(ner_features_matrix)

In [ ]:
# Padronização dos atributos de dependências e NER
scaler = StandardScaler()
dep_features_scaled = scaler.fit_transform(dep_features_matrix)
ner_features_scaled = scaler.fit_transform(ner_features_matrix)

In [ ]:
# Alvo de 3 clusters
num_clusters = 3  # Set the number of clusters

# Cluster usando só estruturas de dependências sintáticas

In [ ]:
kmeans_dependencies = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_dependencies.fit(dep_features_scaled)

# Cria labels para os clusters
clusters_dependencies = kmeans_dependencies.labels_

In [ ]:
# Resultados
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}")
    print(f"Assigned Cluster: {clusters_dependencies[i]}\n")

Text 1: O presidente discursou na ONU.
Assigned Cluster: 2

Text 2: A Apple lançou o novo iPhone no Brasil.
Assigned Cluster: 0

Text 3: O Brasil venceu a Copa América em 2021.
Assigned Cluster: 0

Text 4: A Google está investindo em inteligência artificial.
Assigned Cluster: 1

Text 5: O governo do Brasil anunciou novas medidas econômicas.
Assigned Cluster: 1



# Clusters usando só NER

In [ ]:
kmeans_ner = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_ner.fit(dep_features_scaled)

# Cria labels para os clusters
clusters_ner = kmeans_ner.labels_

# Resultados
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}")
    print(f"Assigned Cluster: {clusters_ner[i]}\n")

Text 1: O presidente discursou na ONU.
Assigned Cluster: 2

Text 2: A Apple lançou o novo iPhone no Brasil.
Assigned Cluster: 0

Text 3: O Brasil venceu a Copa América em 2021.
Assigned Cluster: 0

Text 4: A Google está investindo em inteligência artificial.
Assigned Cluster: 1

Text 5: O governo do Brasil anunciou novas medidas econômicas.
Assigned Cluster: 1



# Cluster usando NER e estruturas sintáticas

In [ ]:
# Combina os atributos advindos das dependências e das NERs
combined_features = np.hstack((dep_features_scaled, ner_features_scaled))

In [ ]:
# Aplica K-means
num_clusters = 3  # Set the number of clusters
kmeans_combined = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_combined.fit(combined_features)

# Cria labels para os clusters
clusters_combined = kmeans_combined.labels_

In [ ]:
# Resultados
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}")
    print(f"Assigned Cluster: {clusters_combined[i]}\n")

Text 1: O presidente discursou na ONU.
Assigned Cluster: 2

Text 2: A Apple lançou o novo iPhone no Brasil.
Assigned Cluster: 0

Text 3: O Brasil venceu a Copa América em 2021.
Assigned Cluster: 0

Text 4: A Google está investindo em inteligência artificial.
Assigned Cluster: 1

Text 5: O governo do Brasil anunciou novas medidas econômicas.
Assigned Cluster: 1



# Enriquecendo a base de NERs

In [ ]:
from spacy.training import offsets_to_biluo_tags

In [ ]:
# Adiciona o NER ao pipeline do spaCy (caso ainda não esteja)
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe("ner")

In [ ]:
TRAINING_DATA = [
    ("A Apple lançou o novo iPhone.", {"entities": [(22, 28, "PRODUCT")]}),
    ("O novo iPhone 12 é o mais recente modelo.", {"entities": [(8, 17, "PRODUCT")]}),
    ("A Samsung apresentou o Galaxy S21.", {"entities": [(23, 33, "PRODUCT")]}),
    ("A Microsoft anunciou o novo Surface Laptop.", {"entities": [(28, 42, "PRODUCT")]}),
    ("A Google lançou o Pixel 5.", {"entities": [(19, 26, "PRODUCT")]}),
    ("O iPad Pro foi apresentado em 2021.", {"entities": [(2, 10, "PRODUCT")]}),
    ("O novo PlayStation 5 será lançado em breve.", {"entities": [(7, 19, "PRODUCT")]})
]

In [ ]:
import random
from spacy.util import minibatch, compounding
from spacy.training import Example

In [ ]:
# Add new labels to the NER
for _, annotations in TRAINING_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [ ]:
# Disable other pipes during training (we only want to train NER)
with nlp.disable_pipes(*[pipe for pipe in nlp.pipe_names if pipe != "ner"]):
    # Initialize the optimizer
    optimizer = nlp.begin_training()

    # Training the model for more epochs
    for iteration in range(100):  # Train for more iterations
        random.shuffle(TRAINING_DATA)
        losses = {}

        # Create batches of training data using minibatch
        batches = minibatch(TRAINING_DATA, size=compounding(4.0, 32.0, 1.001))

        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                try:
                    example = Example.from_dict(doc, annotations)
                except ValueError:
                    print(f"Skipping misaligned entities in: '{text}'")
                    continue
                examples.append(example)

            # Update the model
            nlp.update(examples, sgd=optimizer, drop=0.35, losses=losses)

        print(f"Iteration {iteration} Losses: {losses}")

        # Early stopping condition (stop if losses are minimal or don't improve)
        if losses.get("ner", 0) < 0.01:
            print(f"Early stopping at iteration {iteration}")
            break

# Save the trained model
nlp.to_disk("custom_ner_model")

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "O novo PlayStation 5 será lançado em breve." with entities "[(7, 19, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "A Google lançou o Pixel 5." with entities "[(19, 26, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "O novo iPhone 12 é o mais recente modelo." with entities "[(8, 17, 'PRODUCT')]". 

Iteration 0 Losses: {'ner': np.float32(38.857136)}
Iteration 1 Losses: {'ner': np.float32(47.015873)}
Iteration 2 Losses: {'ner': np.float32(45.02052)}
Iteration 3 Losses: {'ner': np.float32(40.029377)}
Iteration 4 Losses: {'ner': np.float32(31.360744)}
Iteration 5 Losses: {'ner': np.float32(24.362293)}
Iteration 6 Losses: {'ner': np.float32(11.152778)}
Iteration 7 Losses: {'ner': np.float32(7.705338)}
Iteration 8 Losses: {'ner': np.float32(7.768619)}
Iteration 9 Losses: {'ner': np.float32(7.6276407)}
Iteration 10 Losses: {'ner': np.float32(7.6309166)}
Iteration 11 Losses: {'ner': np.float32(6.9190025)}
Iteration 12 Losses: {'ner': np.float32(6.0244155)}
Iteration 13 Losses: {'ner': np.float32(5.951061)}
Iteration 14 Losses: {'ner': np.float32(8.611558)}
Iteration 15 Losses: {'ner': np.float32(4.8120084)}
Iteration 16 Losses: {'ner': np.float32(7.4366217)}
Iteration 17 Losses: {'ner': np.float32(5.5305862)}
Iteration 18 Losses: {'ner': np.float32(3.8421748)}
Iteration 19 Losses: {'ner'

In [ ]:
# Load the trained model
nlp_custom = spacy.load("custom_ner_model")

# Test the model
test_text = "A Apple lançou o novo iPhone."
doc = nlp_custom(test_text)

for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")

Entity: iPhone, Label: PRODUCT


# K-Means usando GPU

In [ ]:
# Verifica GPU
!nvidia-smi

# Instala RAPIDS (versão 22.12 compatível com CUDA 11.2 no Colab)
!wget https://raw.githubusercontent.com/rapidsai/rapidsai-csp-utils/main/colab/rapids-colab.sh
!bash rapids-colab.sh 22.12

Fri Jun  6 10:20:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P0             29W /   70W |    1660MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import numpy as np
from cuml.cluster import KMeans as cuKMeans
from sklearn.preprocessing import StandardScaler

num_clusters = 3  # Set the number of clusters
kmeans_GPU = cuKMeans(n_clusters=num_clusters, random_state=42)
kmeans_GPU.fit(combined_features)

# Get cluster labels
clusters_GPU = kmeans_GPU.labels_

# Display results
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}")
    print(f"Assigned Cluster: {clusters_GPU[i]}\n")


/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


Text 1: O presidente discursou na ONU.
Assigned Cluster: 0

Text 2: A Apple lançou o novo iPhone no Brasil.
Assigned Cluster: 1

Text 3: O Brasil venceu a Copa América em 2021.
Assigned Cluster: 1

Text 4: A Google está investindo em inteligência artificial.
Assigned Cluster: 0

Text 5: O governo do Brasil anunciou novas medidas econômicas.
Assigned Cluster: 2

